In [1]:
import pandas as pd
import numpy as np
import pyodbc

In [2]:
df = pd.read_csv('nyse_holidays.csv')
df = df[df['date'] >= '2020-01-01']
df = df[['date', 'status', 'start_time', 'end_time', 'holiday_name']]

df.rename(columns = {
    "date" : "Date",
    "status" : "Status",
    "start_time" : "Start_Time",
    "end_time" : "End_Time",
    "holiday_name" : "Description"},
    inplace = True)

df['Date'] = pd.to_datetime(df['Date'])

df['Start_Time'] = df['Start_Time'].replace(np.nan, None)
df['End_Time'] = df['End_Time'].replace(np.nan, None)

df.head()

,Date,Status,Start_Time,End_Time,Description
234,2020-01-01,closed,None,None,New Year's Day
235,2020-01-20,closed,None,None,Martin Luther King's Birthday
236,2020-02-17,closed,None,None,Washington's Birthday
237,2020-04-10,closed,None,None,Good Friday
238,2020-05-25,closed,None,None,Memorial Day


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 57 entries, 234 to 290
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         57 non-null     datetime64[ns]
 1   Status       57 non-null     object        
 2   Start_Time   9 non-null      object        
 3   End_Time     9 non-null      object        
 4   Description  57 non-null     object        
dtypes: datetime64[ns](1), object(4)
memory usage: 2.7+ KB


In [4]:
conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=Kiyoworld;"
    "DATABASE=USStocks;"
    "Trusted_Connection=yes;"
)
cursor = conn.cursor()

try :
    insert_query = "INSERT INTO analytics.us_market_holidays ([Date], [Status], Start_Time, End_Time, [Description]) VALUES (?, ?, ?, ?, ?)"
    
    print("Loading Us Market Holiday Data into SQL Server...")
    
    rows = list(df.itertuples(index=False, name=None))
    cursor.executemany(insert_query, rows)
    conn.commit()
    print(f"Successfully inserted {len(df)} records.") 

except Exception as e :
    print(f"Error : {e}")

Loading Us Market Holiday Data into SQL Server...
Successfully inserted 57 records.


In [5]:
import finnhub
finnhub_client = finnhub.Client(api_key="d5q4p8hr01qq2b6afv1gd5q4p8hr01qq2b6afv20")
finnhub_holidays = finnhub_client.market_holiday(exchange='US')

df = pd.DataFrame(finnhub_holidays['data'])
df.reset_index(drop=True, inplace=True)

df[['Start_Time', 'End_Time']] = df['tradingHour'].str.split('-', expand=True)
df['Status'] = np.where(df['tradingHour'].notna() & (df['tradingHour'] != ''), 'short day', 'closed')
df = df[['atDate', 'Status', 'Start_Time', 'End_Time', 'eventName']]

df.rename(columns = {
    "atDate" : "Date",
    "eventName" : "Description"},
    inplace = True)

df['Start_Time'] = df['Start_Time'].apply(lambda x: x if pd.notna(x) and x != '' else None)
df['End_Time'] = df['End_Time'].apply(lambda x: x if pd.notna(x) and x != '' else None)

sorted_df = df.sort_values(by='Date')
sorted_df = sorted_df[sorted_df['Date'] > '2025-01-01']

sorted_df['Date'] = pd.to_datetime(sorted_df['Date'])

In [6]:
try :
    insert_query = "INSERT INTO analytics.us_market_holidays ([Date], [Status], Start_Time, End_Time, [Description]) VALUES (?, ?, ?, ?, ?)"
    
    print("Loading Us Market Holiday Data into SQL Server...")
    
    rows = list(sorted_df.itertuples(index=False, name=None))
    cursor.executemany(insert_query, rows)
    conn.commit()
    print(f"Successfully inserted {len(sorted_df)} records.") 

except Exception as e :
    print(f"Error : {e}")

finally:
    cursor.close()
    conn.close()

Loading Us Market Holiday Data into SQL Server...
Successfully inserted 36 records.
